# Load Training Data

In [68]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# ============================================================
# 0 · imports & configuration
# ============================================================
import boto3, json, pandas as pd, itertools
from tqdm.notebook import tqdm            # Jupyter/HTML bar

s3 = boto3.client("s3")

uris = [
    "s3://echodata25/results/echo-images/nova-pro/gen10-outputs/job_00/koh3yhoh38cy/prompts.jsonl.out",
    "s3://echodata25/results/echo-images/nova-pro/gen10-outputs/job_01/zhnxl9kb6alc/prompts.jsonl.out",
    "s3://echodata25/results/echo-images/nova-pro/gen10-outputs/job_02/h74k0w4wqyuy/prompts.jsonl.out",
    "s3://echodata25/results/echo-images/nova-pro/gen10-outputs/job_03/0db4t3ctdamr/prompts.jsonl.out",
    "s3://echodata25/results/echo-images/nova-pro/gen10-outputs/job_04/9xztwg01glwe/prompts.jsonl.out",
    "s3://echodata25/results/echo-images/nova-pro/gen10-outputs/job_05/9ynxch4xlfeq/prompts.jsonl.out",
]

# ------------------------------------------------------------
# helpers
# ------------------------------------------------------------
def body_iter_lines(uri: str):
    bucket, key = uri.replace("s3://", "", 1).split("/", 1)
    return boto3.client("s3").get_object(Bucket=bucket, Key=key)["Body"].iter_lines()

def row_count(uri: str) -> int:
    "One streaming pass → exact line count."
    return sum(1 for _ in body_iter_lines(uri))

def stream_jsonl(uri: str):
    for raw in body_iter_lines(uri):
        if raw:
            yield json.loads(raw)

# ------------------------------------------------------------
# main loop
# ------------------------------------------------------------
frames = []
outer = tqdm(uris, desc="all jobs", unit="file")   # overall progress

for uri in outer:
    job   = uri.split("/")[-3]
    total = row_count(uri)

    rows = []
    for rec in tqdm(stream_jsonl(uri),
                    total=total,
                    desc=job,
                    unit="rows",
                    leave=True):      # keep each bar
        rows.append(rec)

    frames.append(pd.DataFrame.from_records(rows))
    outer.update()                    # tick the master bar


# final concatenation
df_all = pd.concat(frames, ignore_index=True)
print("Combined shape:", df_all.shape)
# df_all.head()


In [70]:
# ---- Python (works in scripts / Jupyter) ----
import os, psutil, multiprocessing

ram = psutil.virtual_memory()
print(f"RAM total: {ram.total/1e9:.1f} GB   free: {ram.available/1e9:.1f} GB")

cpus_logical  = os.cpu_count()                 #


RAM total: 132.1 GB   free: 98.7 GB


In [71]:
import os, multiprocessing as mp
n_logical  = os.cpu_count()        # includes hyper-threads
n_physical = mp.cpu_count()        # same on Linux; fallback

print(n_logical, "logical cores")


32 logical cores


In [72]:
import pandas as pd, orjson, re
sid_pat = re.compile(r"<SID:([^>]+)>")

def parse_row(row):
    # fast JSON load only if still a string
    mi = orjson.loads(row.modelInput)  if isinstance(row.modelInput,  str) else row.modelInput
    mo = orjson.loads(row.modelOutput) if isinstance(row.modelOutput, str) else row.modelOutput

    # ── SID ──────────────────────────────────────────────────────────────
    sid = None
    for m in mi.get("messages", ()):
        for seg in m.get("content", ()):
            if isinstance(seg, dict):
                m0 = sid_pat.search(seg.get("text", ""))
                if m0:
                    sid = m0.group(1)
                    break
        if sid: break

    # ── conversation text ───────────────────────────────────────────────
    try:
        conv = mo["output"]["message"]["content"][0]["text"]
    except Exception:
        conv = None

    return sid, conv

# process rows lazily; no extra dataframe copies
ids, convs = zip(*map(parse_row, df_all.itertuples(index=False)))

out = pd.DataFrame({"id": ids, "conversations": convs})

In [73]:
out.shape

(266008, 2)

In [74]:
hls_master = pd.read_csv('hls_master_v3.csv')

In [75]:
# out:  cols = ["id", "conversations"]
# hls_master: cols include "DeidentifiedStudyID", "study_dir"

# build a Series once → O(n) memory-light lookup table
study_dir_map = (
    hls_master
    .set_index("DeidentifiedStudyID")["study_dir"]
    .astype(str)          # make sure keys/vals are strings
)

# add column (vectorised; no join-copy)
out["data_source"] = out["id"].map(study_dir_map)


In [76]:
# prerequisites
# pip install boto3 tqdm orjson

import random, boto3, orjson, re, pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# ---------- part 2 · pick one shuf*.mp4 per study --------------
BUCKET   = "echodata25"
ROOT     = "results/echo-images/video-concat"          # constant path prefix
MATCHES  = ("shuf1.mp4", "shuf2.mp4")                  # wanted filenames
MAX_WORKERS = 32                                       # tune for your net-bandwidth / vCPU

s3 = boto3.client("s3")
paginator = s3.get_paginator("list_objects_v2")        # reused → keeps HTTP-pool alive

def find_video(study_id: str, source_dir: str | float) -> str | None:
    """Return s3://…/shuf1|2.mp4 (random pick) or None."""
    if pd.isna(source_dir):
        return None

    prefix = f"{ROOT}/{source_dir}/{study_id}/"        # e.g. results/…/echo-study/<ID>/
    try:
        for page in paginator.paginate(Bucket=BUCKET, Prefix=prefix):
            keys = [obj["Key"] for obj in page.get("Contents", ())
                    if obj["Key"].endswith(MATCHES)]
            if keys:                                   # got one or more candidates
                return f"s3://{BUCKET}/{random.choice(keys)}"
    except s3.exceptions.NoSuchBucket:
        pass                                           # bucket typo guard
    return None

# vectorised parallel lookup with progress bar
ids   = out["id"].values
dirs  = out["data_source"].values

with ThreadPoolExecutor(MAX_WORKERS) as ex:
    out["video"] = list(
        tqdm(ex.map(find_video, ids, dirs), total=len(out), unit="file")
    )

# `out` now has columns: id · conversations · data_source · video


  0%|          | 0/266008 [00:00<?, ?file/s]

In [77]:
# out.head()

In [78]:
! pip install demjson3

In [79]:
new = out.copy()

In [80]:


import re, json, orjson
from tqdm.auto import tqdm

df        = new.copy()                  # keep original safe
COL       = "conversations"

# ────────────────────────────────────────────────────────────────
# 1 · structural typos
# ────────────────────────────────────────────────────────────────
_fix_struct = (
    (re.compile(r'"value="\s*'),                       '"value":"'),
    (re.compile(r'\{"from":"(gpt|human)",\s*""'),      r'{"from":"\1","value":"'),
    (re.compile(r'\{"from":"(gpt|human)"\s*:\s*'),     r'{"from":"\1","value":')
)

def structural(txt: str) -> str:
    for pat, repl in _fix_struct:
        txt = pat.sub(repl, txt)
    return txt


# ────────────────────────────────────────────────────────────────
# 2 · escape control chars & lone back-slashes inside strings
# ────────────────────────────────────────────────────────────────
_str_pat = re.compile(r'"(?:[^"\\]|\\.)*"', re.S)        # every JSON string

def _escape_ctrl(ch: str) -> str:
    """Map control char to JSON escape sequence."""
    if   ch == '\n': return r'\n'
    elif ch == '\r': return r'\r'
    elif ch == '\t': return r'\t'
    else:            return f'\\u{ord(ch):04x}'

_ctrl_pat = re.compile(r'[\x00-\x1F]')                   # 0–31

def escape_in_quotes(txt: str) -> str:
    def patch(m):
        s = m.group(0)
        s = _ctrl_pat.sub(lambda c: _escape_ctrl(c.group(0)), s)
        s = re.sub(r'\\(?!["\\/bfnrtu])', r'\\\\', s)     # lone "\"
        return s
    return _str_pat.sub(patch, txt)


# ────────────────────────────────────────────────────────────────
# 3 · convert raw → Python list   (None if truly truncated)
# ────────────────────────────────────────────────────────────────
def to_list(raw: str):
    try:                         # fast path
        return orjson.loads(raw)
    except orjson.JSONDecodeError:
        fixed = escape_in_quotes(structural(raw))
        try:
            return json.loads(fixed)  # stdlib tolerates pretty well
        except Exception:
            return None               # still broken


# ────────────────────────────────────────────────────────────────
# 4 · clean whole column with a progress bar
# ────────────────────────────────────────────────────────────────
good, bad = [], []
for i, txt in tqdm(enumerate(df[COL]), total=len(df), unit="conv"):
    parsed = to_list(txt)
    if parsed is None:
        bad.append(i)
    else:
        good.append(parsed)

print(f"{len(bad)} rows truncated ➜ {bad[:10]}")

# drop unrecoverable rows, assign cleaned lists
df = df.drop(index=bad).reset_index(drop=True)
df[COL] = good


  0%|          | 0/266008 [00:00<?, ?conv/s]

1213 rows truncated ➜ [366, 1501, 2620, 3930, 5297, 6029, 6868, 7208, 7825, 8270]


In [84]:
# df.head()

In [53]:
raw_df = out.copy()

In [92]:
# # raw_df (or new) → the dataframe that still has the raw strings
# # bad           → list/array of bad-row indices

# def peek_rows(df, indices, n=20, ctx=120):
#     """
#     Print a context slice around the JSON error byte for the first `n` indices.
#     """
#     import orjson
#     for idx in indices[:n]:
#         txt = df.at[idx, "conversations"]
#         try:
#             orjson.loads(txt)            # will raise
#         except orjson.JSONDecodeError as e:
#             pos = e.pos                  # byte offset where parsing broke
#             frag = txt[max(0, pos-ctx): pos+ctx]
#             print(f"\n── row {idx}  (byte {pos}) ──\n{frag}\n")

# peek_rows(raw_df, bad, n=20)

In [86]:
!pip install opencv-python-headless  # lighter, no GUI deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 174.9 MB/s eta 0:00:00a 0:00:01


In [89]:
"""
Compute average frame-count, FPS, and duration over the first `N`
videos listed in df["video"] (each value is an s3://…/shuf*.mp4 URI).

Dependencies  (install once):
    pip install boto3 opencv-python-headless tqdm
"""

import os, tempfile, boto3, cv2, pandas as pd, numpy as np
from urllib.parse import urlparse
from tqdm.auto import tqdm

N        = 20                             # ← how many videos to sample
tmp_dir  = tempfile.mkdtemp()
s3       = boto3.client("s3")

frames, fpss, durs = [], [], []

def download(uri: str, dest_dir: str) -> str:
    """Download S3 object to `dest_dir`, return local path."""
    parsed = urlparse(uri)
    bucket, key = parsed.netloc, parsed.path.lstrip("/")
    local = os.path.join(dest_dir, os.path.basename(key))
    if os.path.exists(local):                      # cached
        return local
    s3.download_file(bucket, key, local)
    return local

for uri in tqdm(df["video"].iloc[:N], total=N, desc="videos"):
    path = download(uri, tmp_dir)

    cap = cv2.VideoCapture(path)
    if not cap.isOpened():                         # skip broken files
        continue
    f = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    r = cap.get(cv2.CAP_PROP_FPS) or np.nan
    cap.release()

    frames.append(f)
    fpss.append(r)
    durs.append(f / r if r else np.nan)

# ── report ──────────────────────────────────────────────────────────────
print(f"sampled videos : {len(frames)}")
print(f"avg frames     : {np.nanmean(frames):.1f}")
print(f"avg fps        : {np.nanmean(fpss):.2f}")
print(f"avg duration s : {np.nanmean(durs):.2f}")


videos:   0%|          | 0/20 [00:00<?, ?it/s]

sampled videos : 20
avg frames     : 3506.2
avg fps        : 30.00
avg duration s : 116.87


In [90]:
len(df)

264795

# Drop Rows Not Yet Processed

1. Also drop all of Syngo (54K rows) until it is reprocessed (findings column)
2. Change all C63 to C61 (RA dilation)

In [95]:
df = df[df["video"].notna() & (df["video"] != "None")]

In [97]:
len(df)

130929

In [100]:
df.iloc[0]

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [101]:
df_ok = df
OUT = "BIMBA-LLaVA-NeXT/echomamba_130k.jsonl"

with open(OUT, "w", encoding="utf-8") as f:
    for _, row in tqdm(df_ok.iterrows(), total=len(df_ok),
                       desc="writing", unit="rows"):
        rec = {
            "id":          str(row["id"]),
            "video":       row["video"],
            "conversations": row["conversations"],   # already a list of dicts
        }

        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"✅ wrote {OUT}")

writing:   0%|          | 0/130929 [00:00<?, ?rows/s]

✅ wrote echomamba_130k.jsonl


In [105]:
import boto3
s3 = boto3.client("s3")

local_file  = "BIMBA-LLaVA-NeXT/echomamba_130k.jsonl"
bucket      = "echodata25"
s3_key      = "data/llava_video_dataset.jsonl"

s3.upload_file(local_file, bucket, s3_key)
print(f"✔️ uploaded to s3://{bucket}/{s3_key}")


✔️ uploaded to s3://echodata25/data/llava_video_dataset.jsonl


In [113]:
rec

{'id': '1.2.276.0.7230010.3.1.2.1714512485.1.1703221347.12348238',
 'video': 's3://echodata25/results/echo-images/video-concat/echo-study/1.2.276.0.7230010.3.1.2.1714512485.1.1703221347.12348238/1.2.276.0.7230010.3.1.3.1714512485.1.1703221347.12348239/shuf2.mp4',
 'conversations': [{'from': 'human',
   'value': '<image>\nGive me a concise first-pass assessment, including any reason for urgent intervention.'},
  {'from': 'gpt',
   'value': '<META_START>\n<C11> <C28> <C35> <C52> <C8>\n<META_END>\n\nThe echocardiogram shows normal left ventricular function, normal valve structures, and no significant abnormalities. There is no urgent intervention required based on these findings.'},
  {'from': 'human',
   'value': 'Can you describe the appearance of the aortic valve in more detail?'},
  {'from': 'gpt',
   'value': 'The aortic valve is tricuspid with slightly thickened cusps, but there is no evidence of aortic stenosis. The valve appears to open and close normally without significant obstr

# Training

In [128]:
import json, pathlib, collections

# path = "BIMBA-LLaVA-NeXT/echomamba_130k.jsonl"   # or the s3-downloaded copy
path = "BIMBA-LLaVA-NeXT/echomamba_130k_clean.jsonl"
problems = collections.Counter()

with open(path) as f:
    for n, line in enumerate(f, 1):
        row = json.loads(line)
        for t in row.get("conversations", []):
            if "value" not in t:                       # LLaVA expects this
                problems["missing value"] += 1
                break
            if not isinstance(t["value"], str):
                problems["not a string"] += 1
                break
        else:
            continue            # all turns in this sample are fine
        print("❌  bad sample @ line", n)              # first few is enough
        if sum(problems.values()) > 20:
            break

print("summary:", problems or "all good")


summary: all good


In [129]:
import boto3
s3 = boto3.client("s3")

local_file  = "BIMBA-LLaVA-NeXT/echomamba_130k_clean.jsonl"
bucket      = "echodata25"
s3_key      = "data/llava_video_dataset_clean.jsonl"

s3.upload_file(local_file, bucket, s3_key)
print(f"✔️ uploaded to s3://{bucket}/{s3_key}")


✔️ uploaded to s3://echodata25/data/llava_video_dataset_clean.jsonl


In [126]:
import json, pathlib

src  = pathlib.Path("BIMBA-LLaVA-NeXT/echomamba_130k.jsonl")
dest = src.with_stem(src.stem + "_clean")           # echomamba_130k_clean.jsonl

bad = 0
with src.open() as fin, dest.open("w") as fout:
    for line in fin:
        row = json.loads(line)
        if any("value" not in t for t in row.get("conversations", [])):
            bad += 1                # skip it
            continue
        fout.write(line)

print(f"✂️  removed {bad} bad rows → {dest}")


✂️  removed 1 bad rows → BIMBA-LLaVA-NeXT/echomamba_130k_clean.jsonl


In [138]:
# from pathlib import Path
# from sagemaker.pytorch import PyTorch
# import sagemaker, boto3

# role = sagemaker.get_execution_role()

# # 👇  we’re already inside the BIMBA folder, so just use cwd()
# SRC_DIR = Path.cwd()                         # /home/.../user-default-efs/BIMBA
# REQ_FILE = SRC_DIR / "BIMBA-LLaVA-NeXT" / "requirements.txt"

# estimator = PyTorch(
#     entry_point      = "train_entrypoint.py",   # file is right here
#     source_dir       = str(SRC_DIR),            # <-- fixed path
#     dependencies     = [str(REQ_FILE)],
#     role             = role,
#     instance_type    = "ml.p4d.24xlarge",
#     instance_count   = 1,
#     framework_version = "2.1",
#     py_version       = "py310",
#     base_job_name    = "bimba-train",
#     disable_profiler = True,
# )

# estimator.fit(
#     inputs = {
#         "training": "s3://echodata25/data/llava_video_dataset_clean.jsonl"
#     },
#     wait   = True,
# )

In [ ]:
from pathlib import Path
from sagemaker.pytorch import PyTorch
import sagemaker, boto3

role = sagemaker.get_execution_role()

# 👇  we’re already inside the BIMBA folder, so just use cwd()
SRC_DIR = Path.cwd()                         # /home/.../user-default-efs/BIMBA
REQ_FILE = SRC_DIR / "BIMBA-LLaVA-NeXT" / "requirements.txt"
IMAGE_URI = "495467399120.dkr.ecr.us-west-2.amazonaws.com/bimba-train:latest"

estimator = PyTorch(
    image_uri        = IMAGE_URI,
    entry_point      = "train_entrypoint.py",   # file is right here
    source_dir       = str(SRC_DIR),            # <-- fixed path
    dependencies     = [str(REQ_FILE)],
    role             = role,
    instance_type    = "ml.p4d.24xlarge",
    instance_count   = 1,
    framework_version = "2.1",
    py_version       = "py310",
    base_job_name    = "bimba-train",
    disable_profiler = True,
)

estimator.fit(
    inputs = {
        "training": "s3://echodata25/data/llava_video_dataset_clean.jsonl"
    },
    wait   = True,
)

[05/11/25 10:08:11] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=549300;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=938834;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[05/11/25 10:10:17] INFO     Creating training-job with name: bimba-train-2025-05-11-10-08-11-012   ]8;id=741955;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=516640;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-05-11 10:10:18 Starting - Starting the training job
2025-05-11 10:10:18 Pending - Training job waiting for capacity......
2025-05-11 10:11:10 Pending - Preparing the instances for training.....................
2025-05-11 10:14:42 Downloading - Downloading input data...
2025-05-11 10:14:57 Downloading - Downloading the training image.....................
2025-05-11 10:18:55 Training - Training image download completed. Training in progress............bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDe

In [139]:
# from pathlib import Path
# from sagemaker.estimator import Estimator
# import sagemaker

# role      = sagemaker.get_execution_role()
# SRC_DIR   = Path.cwd()          # /home/.../BIMBA
# IMAGE_URI = "495467399120.dkr.ecr.us-west-2.amazonaws.com/bimba-train:latest"

# estimator = Estimator(
#     image_uri       = IMAGE_URI,        # ← custom image
#     role            = role,
#     entry_point     = "train_entrypoint.py",
#     source_dir      = str(SRC_DIR),
#     instance_type   = "ml.p4d.24xlarge",
#     instance_count  = 1,
#     base_job_name   = "bimba-train",
#     disable_profiler= True,
# )

# estimator.fit(
#     inputs = {
#         "training": "s3://echodata25/data/llava_video_dataset.jsonl"
#     },
#     wait=True,
# )


[05/11/25 09:40:55] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=98565;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=517303;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              